In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
# matplotlib 한글 꺠짐 방지
# apt-get update
# apt-get install fonts-nanum* 
# apt-get install fontconfig
# fc-cache -fv  # font 캐시 날리기
# rm -rf /home/hy1/.cache/matplotlib/  #matplotliob 폰트 캐시 날리기



In [1]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import datetime as dt




In [2]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [4]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='NanumGothic')

## Load - DataWareHouse

## JDBC DICT

In [8]:
JDBC = {
    'url':'jdbc:oracle:thin:@decorona_high?TNS_ADMIN=/home/big/study/db/Wallet_DECORONA'
    ,'props':{
        'user':'dw_sjs'
       ,'password':'123qwe!@#QWE'
    }   
}

### 1. LOC 테이블 저장

In [4]:
area = spark.read.csv('/corona_data/loc/sido_area.csv', encoding='CP949', header=True)
popu = spark.read.csv('/corona_data/loc/sido_population.csv', encoding='CP949', header=True)

area.show()

+----+------+-----+
| loc|  area|ratio|
+----+------+-----+
|서울|  605 |  0.6|
|부산|  770 |  0.8|
|대구|  883 |  0.9|
|인천| 1065 |  1.1|
|광주|  501 |  0.5|
|대전|  540 |  0.5|
|울산| 1062 |  1.1|
|세종|  465 |  0.5|
|경기|10195 | 10.2|
|강원|16830 | 16.8|
|충북| 7407 |  7.4|
|충남| 8246 |  8.2|
|전북| 8070 |  8.0|
|전남|12348 | 12.3|
|경북|19034 | 19.0|
|경남|10541 | 10.5|
|제주| 1850 |  1.8|
+----+------+-----+



In [9]:
# loc 데이터 저장
area_pop = area.join(popu, on='loc')
area_pop = area_pop.select(col('loc').alias('LOC')
                                   , col('area').alias('AREA')
                                   ,col('total').alias('POPULATION')
                          )
area_pop.show()
area_pop.write.jdbc(url=JDBC['url'], table='LOC', mode='append', properties=JDBC['props'])

+----+------+----------+
| LOC|  AREA|POPULATION|
+----+------+----------+
|서울|  605 |  9736027 |
|부산|  770 |  3396109 |
|대구|  883 |  2412642 |
|인천| 1065 |  3014739 |
|광주|  501 |  1462545 |
|대전|  540 |  1469543 |
|울산| 1062 |  1138419 |
|세종|  465 |   376779 |
|경기|10195 | 13925862 |
|강원|16830 |  1555876 |
|충북| 7407 |  1633472 |
|충남| 8246 |  2181835 |
|전북| 8070 |  1817186 |
|전남|12348 |  1865459 |
|경북|19034 |  2677709 |
|경남|10541 |  3377331 |
|제주| 1850 |   697476 |
+----+------+----------+



### 2. CORONA_PATIENT 테이블 저장

In [6]:
# 코로나 감염자 데이터
path = '/corona_data/patient'
co_patient_json = spark.read.json(path, encoding='UTF-8')
#co_patient_json.show()

+---------------+----------------------+---------+------+----------+--------------+----------+
|_corrupt_record|                 items|numOfRows|pageNo|resultCode|     resultMsg|totalCount|
+---------------+----------------------+---------+------+----------+--------------+----------+
|           null|[{16, 12659, 검역, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|           null|  [{1465, 1461477, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|           null|[{16, 12440, 검역, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|           null| [{1043, 868166, 충...|      500|     1|        00|NORMAL SERVICE|     19684|
|           null|  [{1429, 1214957, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|           null|[{528, 687990, 전남...|      500|     1|        00|NORMAL SERVICE|     19684|
|           null|  [{26161, 22449475...|      500|     1|        00|NORMAL SERVICE|     19684|
|           null|  [{25441, 20983169...|      500|     1|

In [8]:
data = []
for r1 in co_patient_json.select('items').toLocalIterator():
    if not r1.items:
        continue
    for r2 in r1.items:
        data.append(r2)

patient_data = spark.createDataFrame(data)
patient_data.show(3)

co_patients = patient_data.select(
    patient_data.gubun.alias('LOC')
    ,patient_data.deathCnt.alias('DEATH_CNT')
     ,patient_data.defCnt.alias('DEF_CNT')
     ,patient_data.localOccCnt.alias('LOC_OCC_CNT')
     ,patient_data.qurRate.alias('QUR_RATE')
     ,patient_data.stdDay.alias('STD_DAY')
).where(~(col('LOC').isin(['합계','검역'])) ).distinct()
co_patients.printSchema()


+--------+-------+-----+-------+-----------+------+------------+----------+-----------+-----------+-------+----------+
|deathCnt| defCnt|gubun|gubunCn|    gubunEn|incDec|isolClearCnt|isolIngCnt|localOccCnt|overFlowCnt|qurRate|    stdDay|
+--------+-------+-----+-------+-----------+------+------------+----------+-----------+-----------+-------+----------+
|      16|  12659| 검역| 隔離區|  Lazaretto|    17|           0|         0|          0|         17|      -|2022-08-25|
|      16|  12659| 검역| 隔離區|  Lazaretto|    17|           0|         0|          0|         17|      -|2022-08-25|
|    6715|6109867| 경기|   京畿|Gyeonggi-do| 27032|           0|         0|      27007|         25|  45040|2022-08-25|
+--------+-------+-----+-------+-----------+------+------------+----------+-----------+-----------+-------+----------+
only showing top 3 rows

root
 |-- LOC: string (nullable = true)
 |-- DEATH_CNT: string (nullable = true)
 |-- DEF_CNT: string (nullable = true)
 |-- LOC_OCC_CNT: string (nullable =

In [10]:
co_patients.write.jdbc(url=JDBC['url'], table='CORONA_PATIENTS', mode='append', properties=JDBC['props'])

### 3.  CORONA_VACCINE 테이블 저장

In [3]:
file_name = '/corona_data/vaccine/'
vaccine = spark.read.json(file_name, multiLine=True)
vaccine.show(3)

+----------------------+-----------------------------+
|                  data|                         meta|
+----------------------+-----------------------------+
|[{서울, 8,340,483, ...|{{지역, 1차접종자수, 2차접...|
+----------------------+-----------------------------+



In [8]:
vaccine.select('meta').collect()

[Row(meta=Row(cols=Row(loc='지역', v1='1차접종자수', v2='2차접종자수', v3='3차접종자수', v4='4차접종자수'), desc='지역별 코로나 예방접종 인구 현황', std_day='2022-09-14'))]

In [17]:
data = []
for r1 in vaccine.select(vaccine.data, vaccine.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        # row 객체를 만들기 위해 dict 생성
        # ** 파이썬 압축해제 키워드 
        # **kwargs   => 여러 쌍의 키워드 args 가 넘어오면 받아서 dictionary로 반환
        # fnc(**dict) => dict에 있는 key-value 값들이 여러쌍의 kwargs  형태로 함수에 전달
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day']
        data.append(Row(**temp))
        
vaccine_data = spark.createDataFrame(data)
vaccine_data.show(3)

    

+----+---------+---------+---------+---------+----------+
| loc|       v1|       v2|       v3|       v4|   std_day|
+----+---------+---------+---------+---------+----------+
|서울|8,340,483|8,261,426|6,057,235|1,224,104|2022-09-14|
|부산|2,879,873|2,850,933|2,134,510|  488,021|2022-09-14|
|대구|2,019,298|1,996,340|1,402,588|  259,143|2022-09-14|
+----+---------+---------+---------+---------+----------+
only showing top 3 rows



In [19]:
# column을 row로 : stack
# row를 column : pivot
pd_vaccine = vaccine_data.to_pandas_on_spark()
pd_vaccine = pd_vaccine.set_index(['loc','std_day'])
pd_vaccine = pd_vaccine.stack()
pd_vaccine = pd_vaccine.to_dataframe('V_CNT')
pd_vaccine

V_CNT
loc std_day                  
서울  2022-09-14 v2   8,261,426
               v4   1,224,104
               v1   8,340,483
               v3   6,057,235
부산  2022-09-14 v2   2,850,933
               v4     488,021
               v1   2,879,873
               v3   2,134,510
대구  2022-09-14 v2   1,996,340
               v4     259,143
               v1   2,019,298
               v3   1,402,588
인천  2022-09-14 v2   2,546,047
               v4     393,704
               v1   2,570,612
               v3   1,907,447
광주  2022-09-14 v2   1,251,020
               v4     233,179
               v1   1,262,691
               v3     968,031
대전  2022-09-14 v2   1,233,937
               v4     192,896
               v1   1,246,229
               v3     907,539
울산  2022-09-14 v2     959,230
               v4     124,077
               v1     968,596
               v3     719,468
세종  2022-09-14 v2     294,828
               v4      34,536
               v1     298,598
               v3     209,144
경기  2022-09-14 v2  11,711,691
               v4   1,677,395
               v1  11,827,683
               v3   8,676,645
강원  2022-09-14 v2   1,338,892
               v4     268,701
               v1   1,350,278
               v3   1,061,985
충북  2022-09-14 v2   1,416,869
               v4     276,875
               v1   1,429,541
               v3   1,105,794
충남  2022-09-14 v2   1,881,763
               v4     364,976
               v1   1,899,464
               v3   1,475,425
전북  2022-09-14 v2   1,580,503
               v4     369,772
               v1   1,593,725
               v3   1,284,072
전남  2022-09-14 v2   1,640,035
               v4     443,605
               v1   1,654,730
               v3   1,363,151
경북  2022-09-14 v2   2,269,348
               v4     394,082
               v1   2,293,313
               v3   1,724,411
경남  2022-09-14 v2   2,854,105
               v4     482,797
               v1   2,883,530
               v3   2,142,029
제주  2022-09-14 v2     584,795
               v4      91,773
               v1     590,570
               v3     437,768

In [20]:
pd_vaccine = pd_vaccine.reset_index()
pd_vaccine = pd_vaccine.rename(columns={'level_2':'V_TH'})
vaccine_data = pd_vaccine.to_spark()


In [21]:
vaccine_data.drop('level_0')
vaccine_data.drop('index')
vaccine_data.show(3)

+----+----------+----+---------+
| loc|   std_day|V_TH|    V_CNT|
+----+----------+----+---------+
|서울|2022-09-14|  v2|8,261,426|
|서울|2022-09-14|  v4|1,224,104|
|서울|2022-09-14|  v1|8,340,483|
+----+----------+----+---------+
only showing top 3 rows



In [ ]:
vaccine_data.write.jdbc(url=JDBC['url'], table='CORONA_VACCINE', mode='append', properties=JDBC['props'])

### 4. LOC_FACILITY_CNT	테이블 저장